###基本步驟

In [1]:
# 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#%cd /content/drive/MyDrive/團體專題Linebot/test6

In [3]:
#%cd /content/drive/MyDrive/團體專題Linebot/test4

In [4]:
# %cd /content/drive/MyDrive/Tibame_ai工程師班/期末專題/LineBot/Colab/IKnowFish1024

In [5]:
 %cd /content/drive/MyDrive/AI Linebot 期末專題/IKnowFish1101

/content/drive/MyDrive/AI Linebot 期末專題/IKnowFish1101


In [6]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok pyngrok mysql-connector pyimgur
!pip install --upgrade google-cloud-storage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.7 MB/s 
     |████████████████████████████████| 745 kB 28.4 MB/s 
     |████████████████████████████████| 11.9 MB 49.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=4158a799d0601fbb599f1795301fc1029987c38097a00cb39745334f2680cbcd
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp37-cp37m-linux_x86_64.whl size=247965 sha256=d27fa3591657898e69f99a6346078ae028482c5b84481ed0b192221e94c37aca
  Stored in directory: /root/.cache/pip/wheels/42/2f/c3/692fc7fc1f0d8c06b9175d94f0fc30f4f92348f5df5af1b8b7
  Created wheel for pyimgur: filename=pyimgur-0.6.0-py3-none-any.whl size=32437 sha256=61336d099387224bb4d068f95e03d577e3555b3799212e417518a488b280723f
  Stored in directory: /root/.

In [7]:
# export GOOGLE_APPLICATION_CREDENTIALS="/content/drive/MyDrive/團體專題Linebot/test6/servicekey_googlecloud.json"

In [8]:
# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)
# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import *
import requests
import os
from google.cloud import storage
from detection.detection import Detector
import pyimgur
from PIL import Image


In [9]:
with open('env.json') as f:
    env = json.load(f)
with open('fish_intro.json') as f:
    fish_intro_dict = json.load(f)

In [10]:
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi(env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])

path = env["YOUR_MAIN_PATH"]

In [11]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body:" + body)
    print(body)

    # 記錄用戶log
    f = open(f"{path}/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

###handler-FollowEvent

In [12]:
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open(f"{path}/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')

    #line_bot_api.link_rich_menu_to_user(event.source.user_id, rich_menu_id)

    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('非常感謝您的關注!')]
    )

###情境互動連接

In [13]:
# 建立Button Template

story_button1 = TemplateSendMessage(
    alt_text="Buttons template",
    template=ButtonsTemplate(
        title="歡迎來到海科館, 接下來請依照提示完成任務, 完成後將能夠獲得神秘小禮物",
        text="準備好要開啟海科館大冒險了嗎",
        thumbnail_image_url="https://imgur.com/Sth8TjQ.png",
        actions=[
            {"type":"message",
            "label":"我準備好了",
            "text":"任務開始!",
            },
            {"type":"message",
            "label":"小等一下",
            "text":"等你喔",
            }
        ]
    )
)

In [14]:
hint = "\
    @麵包海星\n\
    @飯匙鯊\n\
    @花園鰻小子\n\
    @獅子魚"

story_dict = {
    "script_a1": "阿尼是一隻小丑魚，鮮豔的橘色搭配白色美麗的條紋看起來帥氣無比。\n不過有件事情一直煩惱著他...就是無論他怎麼努力都交不到女朋友QQ\n聽說今天在智能海洋館裡面有隻美麗的魚也正在尋找自己的伴侶，現在就讓我們來協助阿尼找到真愛吧",
    "script_a2": "一進入館內的第一個區域「潮境之濱」，阿尼發現自己置身美麗的3D投影當中...這時阿尼突然聽到一個低沈的聲音從頭上傳來...\n哇！原來是大鯨魚在說話！",
    "script_d1": "花園鰻小子：\n什麼...想不到你這麼厲害，竟然可以找到我...\n那天我經過海星婆婆的家，聞到一陣陣香味，剛好我的肚子好餓好餓，所以才會偷偷把湯喝掉的，我會親自去和麵包海星婆婆道歉的，真是抱歉！",
    "script_e1": "獅子魚：你就是尼莫對吧，你幫助麵包海星婆婆的事已經傳片整個海科館了呢！像你這麼善良聰明又帥氣的魚，能夠和你當朋友真是太好了！\n在阿尼與獅子魚與見面後兩人立刻墜入愛河，從此過著幸福又快樂的生活，阿尼為了感謝您的協助，在此提供一張海科館的八折優惠券作為報答，現在就到販賣區挑一個喜歡的紀念品或是前往互動體驗區展開全新冒險吧！",
    "task_a": "大鯨魚：\n看你帥氣的外表，你就是阿尼對吧...想要找到女朋友的話你可以問問我們館內的媒人婆「麵包海星」婆婆，什麼...你問我她看起來是什麼樣子嗎？\n「麵包海星」一般為5隻腕足，但腕足特別粗短，區分不明顯看起來胖胖的就像是菠蘿麵包一樣喔哈哈哈，她就住在接下來的「潮境之魚」區域當中，我相信他會給你許多有用的資訊的！",
    "task_b": "「麵包海星」婆婆：\n小伙子我正在忙呢！找我有什麼事啊？\n什麼...你想要找女朋友？正好我知道館內有隻美麗的魚也在尋找伴侶呢！\n不過...我昨天剛煮好的濃湯被偷喝了，我現在正忙著找出小偷呢！如果你可以幫忙我的話，我就幫你找到女朋友作為回報！\n現在你有看到旁邊的七公尺超大魚缸嗎，我要你去和「飯匙鯊」警長報案，讓他揪出可惡的小偷！",
    "task_c": "「飯匙鯊」警長：\n是麵包海星婆婆請你來報案的啊...麵包海星婆婆的濃湯是最好喝的！想不到竟然會發生這樣的事...真是可惡！\n依現場殘留的沙子看來，小偷應該是某種會躲藏在沙子裡的生物，而要能夠從門縫進入，想必小偷一定是個苗條的傢伙呢！\n我想小偷應該是走不遠的...也許在接下來的「潮下視界」就可以找他了！",
    "task_d": "麵包海星婆婆：\n做得好阿尼！花園鰻小子已經來和我道歉過並且答應幫我做一個月的家事來補償！\n為了回報你的辛苦我就告訴你吧，在「潮水彩繪」區域中的「獅子魚」小姐正在找尋伴侶！\n獅子魚小姐的外表可是漂亮的喔，美麗的線條斑紋搭配放射狀的胸鰭和背鰭，可是迷倒眾魚呢，不過千萬要小心她可是有毒性的喔！一般人可別離她太近！但我想他會喜歡你這個帥氣又聰明的小帥哥的！\n現在就去找她吧，和她說你是麵包海星婆婆認證的好魚，至於後續就得看你的表現了，好好加油阿尼！",
    "hint": f"請依據對話中的線索，拍攝正確物種之照片並傳送於對話筐內\n(若是沒有找到正確魚類的可以再次上傳照片嘗試喔)\n\n#現行先用<numstQuestion>@<species>來代替圖片\nex:\n{hint}",
    "麵包海星": "麵包海星婆婆有「5隻特別粗短的腕足，看起來胖胖像是菠蘿麵包」請再找找～",
    "飯匙鯊": "飯匙鯊頭部前圓而寬闊，頭與胸鰭非常容易區別，他就在七尺大魚缸中游泳呢！請再找找～",
    "花園鰻小子": "根據線索花園鰻小子小偷是個苗條的傢伙，常常會躲在沙子裡面，而且跟線報得知他的身上有著明顯的黑色斑點，請再找找吧～",
    "獅子魚": "這不是獅子魚小姐啦～\n獅子魚擁有美麗的線條斑紋搭配放射狀的胸鰭和背鰭，請再找找看吧",
    "a": "麵包海星",
    "b": "飯匙鯊",
    "c": "花園鰻小子",
    "d": "獅子魚",
}

In [15]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
story_quick_reply_start = QuickReplyButton(
    image_url="https://imgur.com/Wiy5swy.png",
    action=PostbackAction(
        label="故事開始",
        display_text="故事開始",
        data="start"
    )
)

story_quick_reply_script_a1 = QuickReplyButton(
    image_url="https://imgur.com/aU2rYP5.png",
    action=PostbackAction(
        label="進入「潮境之濱」",
        display_text="進入「潮境之濱」",
        data="script_a1"     
    )
)
story_quick_reply_script_a2 = QuickReplyButton(
    image_url="https://imgur.com/FLm61kB.png",
    action=PostbackAction(
        label="與大鯨魚對話",
        display_text="與大鯨魚對話",
        data="script_a2"
    )
)

story_quick_reply_script_d1 = QuickReplyButton(
    "https://imgur.com/475zKbB.png",
    action=PostbackAction(
        label="再訪麵包海星婆婆",
        display_text="再訪麵包海星婆婆", 
        data="script_d1"
    )
)
story_quick_reply_script_e1 = QuickReplyButton(
    "https://imgur.com/ekd95iS.png",
    action=PostbackAction(
        label="終於...見到「獅子魚」小姐",
        display_text="終於...見到「獅子魚」小姐", 
        data="script_e1"
    )
)
story_quick_reply_task_a = QuickReplyButton(
    "https://imgur.com/x1Rccvl.png",
    action=PostbackAction(
        label="進入「潮境之魚」，尋找「麵包海星」婆婆",
        display_text="進入「潮境之魚」，尋找「麵包海星」婆婆",
        data="hint"
    )
)
story_quick_reply_task_b = QuickReplyButton(
    "https://imgur.com/k3mBJqm.png",
    action=PostbackAction(
        label="尋找「飯匙鯊」警長",
        display_text="尋找「飯匙鯊」警長",
        data="hint"
    )
)
story_quick_reply_task_c = QuickReplyButton(
    "https://imgur.com/DxMLo93.png",
    action=PostbackAction(
        label="進入「潮下視界」，尋找小偷", 
        display_text="進入「潮下視界」，尋找小偷",
        data="hint"
    )
)
story_quick_reply_task_d = QuickReplyButton(
    "https://imgur.com/b25kLsC.png",
    action=PostbackAction(
        label="進入「潮水彩繪」，尋找「獅子魚」小姐", 
        display_text="進入「潮水彩繪」，尋找「獅子魚」小姐",
        data="hint"
    )
)

# 相機 & 相簿
# cameraQuickReplyButton = QuickReplyButton(
#     action=URIAction(
#         label="打開相機",
#         uri="https://line.me/R/nv/camera/"
#     )
# )
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="打開相機")
)
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

In [16]:
story_task_dict = {
    "a": story_quick_reply_task_a,
    "b": story_quick_reply_task_b,
    "c": story_quick_reply_task_c,
    "d": story_quick_reply_task_d
}

### handler-MessageEvent-TextMessage

In [17]:
# 建立觸發回覆

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)   
    user_profile_dict = vars(user_profile)
    try:
        with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "r") as f:
            last_line = f.readlines()[-1]
            user_profile_dict = json.loads(last_line)

    except:
        pass

    if event.message.text == "開啟情境互動任務":
        line_bot_api.reply_message(event.reply_token, 
                                story_button1)
    if event.message.text == "任務開始!":
        if not os.path.exists("users_info"):
            os.makedirs("users_info")
        
        user_profile_dict["story_stage"] = "a"
        with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as myfile:
            myfile.write(json.dumps(user_profile_dict,sort_keys=True))
            myfile.write('\n')

        
        flex_message = TextSendMessage(
            text="請點擊下方泡泡提示進行故事",
            quick_reply=QuickReply(
                items=
                [
                    story_quick_reply_start
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    if event.message.text[0] == "@":
        stage = user_profile_dict["story_stage"] if user_profile_dict.get("story_stage", False) else ""

        info = event.message.text.split("@")
        if stage == "a" and info[1] == "麵包海星":
            user_profile_dict["story_stage"] = "b"
            with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as f:
                f.write(json.dumps(user_profile_dict,sort_keys=True))
                f.write('\n')

            flex_message = TextSendMessage(
                text=story_dict["task_b"]
                )

            image_message = ImageSendMessage(
                original_content_url="https://imgur.com/uyxeCsK.png",
                preview_image_url="https://imgur.com/uyxeCsK.png",
                quick_reply=QuickReply(
                    items=[
                    story_quick_reply_task_b
                    ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [   
                    flex_message,
                    image_message
                ]
            )
        elif stage == "b" and info[1] == "飯匙鯊":
            user_profile_dict["story_stage"] = "c"
            with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as f:
                f.write(json.dumps(user_profile_dict,sort_keys=True))
                f.write('\n')
            flex_message = TextSendMessage(
                text=story_dict["task_c"]
            )
            image_message = ImageSendMessage(
                original_content_url="https://imgur.com/8Bl4F7r.png",
                preview_image_url="https://imgur.com/8Bl4F7r.png",
                quick_reply=QuickReply(
                    items=[
                    story_quick_reply_task_c
                    ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message,
                    image_message
                ]
            )
        elif stage == "c" and info[1] == "花園鰻小子":
            user_profile_dict["story_stage"] = "d"
            with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as f:
                f.write(json.dumps(user_profile_dict,sort_keys=True))
                f.write('\n')
            flex_message = TextSendMessage(
                text=story_dict["script_d1"]
            )
            image_message = ImageSendMessage(
                original_content_url="https://imgur.com/CkeeWqm.png",
                preview_image_url="https://imgur.com/CkeeWqm.png",
                quick_reply=QuickReply(
                    items=[
                    story_quick_reply_script_d1
                    ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message,
                    image_message
                ]
            )
        elif stage == "d" and info[1] == "獅子魚":
            user_profile_dict["story_stage"] = "e"
            with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as f:
                f.write(json.dumps(user_profile_dict,sort_keys=True))
                f.write('\n')
            flex_message = TextSendMessage(
                text="獅子魚小姐: 你找到我啦～"         
            )
            image_message = ImageSendMessage(
                original_content_url="https://imgur.com/ci4RYi9.png",
                preview_image_url="https://imgur.com/ci4RYi9.png",
                quick_reply=QuickReply(
                    items=[
                    story_quick_reply_script_e1
                    ]
            )
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message, 
                    image_message
                ]
            )
        elif stage in ["a", "b", "c", "d"]:
            flex_message = TextSendMessage(
                text=f"這不是{story_dict[stage]}喔，" + story_dict[story_dict[stage]],
                quick_reply=QuickReply(
                    items=
                    [
                        story_task_dict[stage]
                    ]
                )
        )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    flex_message
                ]
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                [TextSendMessage("請點選\n\t圖文選單->情境互動\n開始新的旅程喔!\n")]
            )
            

###準備ImageEvent要用到的函式

In [18]:
# 透過雲端連結金鑰建立客戶端
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "servicekey_googlecloud.json"
storage_client = storage.Client()
#定義函式-將圖片傳到雲端
def upload_blob(bucket_name, source_file_name, destination_blob_name):
  bucket = storage_client.bucket(bucket_name)
  blob = bucket.blob(destination_blob_name)
  blob.upload_from_filename(source_file_name)
  print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# model引入
class_name_path="fish_classes.txt"
weights_path="yolov4_weights_1021_best_3.h5"
model = Detector(class_name_path=class_name_path, weights_path=weights_path)
model.set_iou_threshold(0.3)
model.set_score_threshold(0.45)
model.build_model()
model.load_weights()

# 定義函式-將圖片變成連結
def glucose_graph(client_id, imgpath):
	im = pyimgur.Imgur(client_id)
	upload_image = im.upload_image(imgpath, title="Uploaded with PyImgur")
	return upload_image.link

nms iou: 0.3 score: 0.45


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


###handler-MessageEvent-ImageEvent

In [19]:
@handler.add(MessageEvent, message=ImageMessage)
# 收到照片後將照片暫存在本地端
def handle_image_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_profile_dict = vars(user_profile)
    
    try:
        with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "r") as f:
            last_line = f.readlines()[-1]
            user_profile_dict = json.loads(last_line)
            detection = user_profile_dict["detection"]

    except:
        detection = ""

    img_content = line_bot_api.get_message_content(event.message.id)
    with open (f"{event.message.id}.jpg", "wb") as fd:
        for chunk in img_content.iter_content():
            fd.write(chunk)

    # 然後丟到CloudStorage上    
    upload_blob("iknowfish_img", 
                f"{event.message.id}.jpg", 
                f"imgdirectory/{event.message.id}.jpg")
    
    # 模型預測
    if detection == "on":
        output_img, info = model.detect(f"{event.message.id}.jpg")
        # 並將預測好的圖片回傳給用戶
        im = Image.fromarray(output_img)
        im.save("to_user_img.jpg")
        img_url = glucose_graph("6a0e0c8afe17efb", "to_user_img.jpg")
        
        os.remove("to_user_img.jpg")
        os.remove(f"{event.message.id}.jpg")

        user_profile_dict["detection"] = "off"
        with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as myfile:
            myfile.write(json.dumps(user_profile_dict,sort_keys=True))
            myfile.write('\n')

        if info:
            fishes = set()
            introductions = ""

            for _, v in info.items():
                fishes.add(v["class_name"])
            for fish in fishes:
                introductions += (fish + ":\n" + fish_intro_dict[fish] + "\n")

            image_message = ImageSendMessage(
                    original_content_url=img_url, 
                    preview_image_url=img_url
            )

            flex_message = TextSendMessage(
                    introductions
            )
            line_bot_api.reply_message(
                event.reply_token,
                [
                    image_message,
                    flex_message
                ]
            )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(
                    "很抱歉沒有偵測到魚類"
                )
            )
    else: 
        os.remove(f"{event.message.id}.jpg")
        line_bot_api.reply_message(
            event.reply_token, 
            TextSendMessage(
                "劇情模式的辨識系統還沒上線喔~\n\
                請使用@來遊玩喔~\n\
                e.g. @麵包海星"
            )
        )

###handler-PostbackEvent

In [20]:
@handler.add(PostbackEvent)
def handle_postback_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_profile_dict = vars(user_profile)
    try:
        with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "r") as f:
            last_line = f.readlines()[-1]
            user_profile_dict = json.loads(last_line)

    except:
        pass

    if event.postback.data == 'rich_menu_others':
        flex_message = TextSendMessage(
            text='最新活動及魚種介紹',
            quick_reply=QuickReply(
                items=[
                QuickReplyButton(image_url="https://imgur.com/OzwYTZb.png", action=MessageAction(label="其他活動", text="其他活動")),
                QuickReplyButton(image_url="https://imgur.com/htAimCQ.png", action=MessageAction(label="魚種介紹", text="魚種介紹" )),
                QuickReplyButton(image_url="https://imgur.com/C6DM4F7.png", action=URIAction(
                    label='進入官網', 
                    
                    uri='https://www.nmmst.gov.tw/chhtml/'
                ))]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "start":
        flex_message = TextSendMessage(
            text=story_dict["script_a1"]
            
        )
        image_message = ImageSendMessage(
            original_content_url="https://imgur.com/4zX3C4z.png", 
            preview_image_url="https://imgur.com/4zX3C4z.png",
            quick_reply=QuickReply(
                items=[
                story_quick_reply_script_a1
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message,
                image_message
            ]
        )
    elif event.postback.data == "script_a1":
        flex_message = TextSendMessage(
            text=story_dict["script_a2"]
            
        )
        image_message = ImageSendMessage(
            original_content_url="https://imgur.com/jjafu0x.png", 
            preview_image_url="https://imgur.com/jjafu0x.png",
            quick_reply=QuickReply(
                items=[
                story_quick_reply_script_a2
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message,
                image_message
            ]
        )
    elif event.postback.data == "script_a2":
        flex_message = TextSendMessage(
            text=story_dict["task_a"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_task_a
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "script_d1":
        flex_message = TextSendMessage(
            text=story_dict["task_d"],
            quick_reply=QuickReply(
                items=[
                story_quick_reply_task_d
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "script_e1":
        line_bot_api.reply_message(
            event.reply_token,
            [
                TextSendMessage(story_dict["script_e1"])
            ]
        )
    elif event.postback.data == "hint":
        flex_message = TextSendMessage(
            text=story_dict["hint"],
            quick_reply=QuickReply(
                items=[
                cameraQuickReplyButton,
                cameraRollQRB
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )
    elif event.postback.data == "fish_detection":
        if not os.path.exists("users_info"):
            os.makedirs("users_info")
        
        user_profile_dict["detection"] = "on"
        with open("{}/users_info/{}.txt".format(path, user_profile_dict["user_id"]), "a") as myfile:
            myfile.write(json.dumps(user_profile_dict,sort_keys=True))
            myfile.write('\n')

        flex_message = TextSendMessage(
            text="拍一張相片或是從圖庫上傳照片來進行魚類辨識",
            quick_reply=QuickReply(
                items=[
                cameraQuickReplyButton,
                cameraRollQRB
                ]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )

In [21]:
if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://692c-34-74-6-155.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"Ud1a0211c95ad00e5ebf1a88cee4dbc69","events":[]}


INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 13:41:41] "POST / HTTP/1.1" 200 -


{"destination":"Ud1a0211c95ad00e5ebf1a88cee4dbc69","events":[{"type":"postback","postback":{"data":"fish_detection"},"webhookEventId":"01GGW97DWSMP21MDQKJXHJN217","deliveryContext":{"isRedelivery":false},"timestamp":1667396515401,"source":{"type":"user","userId":"U33c0af05ef3c67fec03857ef495fb0ab"},"replyToken":"04cc32c5c53048248f8838d442159563","mode":"active"}]}


...

INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 13:44:33] "POST / HTTP/1.1" 200 -


{"destination":"Ud1a0211c95ad00e5ebf1a88cee4dbc69","events":[{"type":"postback","postback":{"data":"script_e1"},"webhookEventId":"01GGW9CDBX0MZBRMZRXZ9PED6V","deliveryContext":{"isRedelivery":false},"timestamp":1667396678536,"source":{"type":"user","userId":"U33c0af05ef3c67fec03857ef495fb0ab"},"replyToken":"0b4d3fbc8ec442f0a838eb4787b30710","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 13:44:40] "POST / HTTP/1.1" 200 -


{"destination":"Ud1a0211c95ad00e5ebf1a88cee4dbc69","events":[{"type":"postback","postback":{"data":"rich_menu_others"},"webhookEventId":"01GGW9CYGR7NE4B3FXB0Z8C8KD","deliveryContext":{"isRedelivery":false},"timestamp":1667396696306,"source":{"type":"user","userId":"U33c0af05ef3c67fec03857ef495fb0ab"},"replyToken":"024b29c2a0ea4c1199e9728389b37161","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [02/Nov/2022 13:44:58] "POST / HTTP/1.1" 200 -


In [22]:
# !npx kill-port 5000